In [2]:
import hoomd
import numpy
import hoomd.md
import sys
import gsd
import gsd.hoomd
hoomd.context.initialize("")

HOOMD-blue 2.3.3 DOUBLE HPMC_MIXED TBB SSE SSE2 
Compiled: 07/04/2018
Copyright 2009-2018 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU


In [3]:
import mbuild as mb
box = mb.load("/home/themotivation/Avo_atoms/airHCN.pdb")
box.save("/home/themotivation/Avo_atoms/airHCN.gsd")
print("it worked!")

OSError: /home/themotivation/Avo_atoms/airHCN.gsd exists; not overwriting

In [4]:
system = hoomd.init.read_gsd("/home/themotivation/Avo_atoms/airHCN.gsd")

notice(2): Group "all" created containing 19904 particles


In [5]:
#first line opens the gsd file so hoomd can read it. second line specifies the first (and only) frame of the gsd file (this gsd . 
#third line  confirms frame. 
#fourth line shows us the type/name of particles (atoms/molecules) that hoomd recognizes
#fifth line shows us the snapshot at the specified timestep in the gsd file

t = gsd.hoomd.open(name='/home/themotivation/Avo_atoms/airHCN.gsd', mode='rb')
snap = t[0]
snap.configuration.step
print(snap.particles.types)
print(snap.particles.position)
#snap.particles.particle_types
#snap.particles.position

['AR', 'C', 'H', 'N', 'O']
[[-15.002001  17.577    -15.713   ]
 [-15.654     16.912    -15.264   ]
 [ -0.092    -39.628     -9.145   ]
 ...
 [ 39.076    -38.936    -39.873   ]
 [ 39.579998 -39.948    -39.989   ]
 [ 38.61     -38.001    -39.765   ]]


In [6]:
snap2 = hoomd.data.make_snapshot(N=19904, box=hoomd.data.boxdim(L=100), particle_types=['AR', 'C', 'H', 'N', 'O'])
for x in range(0, 19904):
    snap2.particles.position[x] = snap.particles.position[x]

print(snap2.particles.position)

[[-15.002001  17.577    -15.713   ]
 [-15.654     16.912    -15.264   ]
 [ -0.092    -39.628     -9.145   ]
 ...
 [ 39.076    -38.936    -39.873   ]
 [ 39.579998 -39.948    -39.989   ]
 [ 38.61     -38.001    -39.765   ]]


In [7]:
print(snap2.particles.position)

[[-15.002001  17.577    -15.713   ]
 [-15.654     16.912    -15.264   ]
 [ -0.092    -39.628     -9.145   ]
 ...
 [ 39.076    -38.936    -39.873   ]
 [ 39.579998 -39.948    -39.989   ]
 [ 38.61     -38.001    -39.765   ]]


In [8]:
print(snap2.particles.velocity)

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [9]:
hoomd.init.read_snapshot(snap2);

**ERROR**: Cannot initialize more than once


RuntimeError: Error initializing

In [10]:
nl = hoomd.md.nlist.cell();

In [11]:
lj = hoomd.md.pair.lj(r_cut=2.5, nlist=nl);

In [12]:
lj.pair_coeff.set('N','N', epsilon=1.0, sigma=1.0);
lj.pair_coeff.set('N','0', epsilon=1.0, sigma=1.0);
lj.pair_coeff.set('N','AR', epsilon=1.0, sigma=1.0);
lj.pair_coeff.set('N','H', epsilon=1.0, sigma=1.0);
lj.pair_coeff.set('N','C', epsilon=1.0, sigma=1.0);

lj.pair_coeff.set('O','O', epsilon=1.0, sigma=1.0);
lj.pair_coeff.set('O','AR', epsilon=1.0, sigma=1.0);
lj.pair_coeff.set('O','C', epsilon=1.0, sigma=1.0);
lj.pair_coeff.set('O','H', epsilon=1.0, sigma=1.0);

lj.pair_coeff.set('AR','AR', epsilon=1.0, sigma=1.0);
lj.pair_coeff.set('AR','H', epsilon=1.0, sigma=1.0);
lj.pair_coeff.set('AR','N', epsilon=1.0, sigma=1.0);
lj.pair_coeff.set('AR','C', epsilon=1.0, sigma=1.0);

lj.pair_coeff.set('H','C', epsilon=1.0, sigma=1.0);
lj.pair_coeff.set('C','C', epsilon=1.0, sigma=1.0);
lj.pair_coeff.set('H','H', epsilon=1.0, sigma=1.0);
lj.pair_coeff.set('N','O', epsilon=1.0, sigma=1.0);

In [13]:
hoomd.md.integrate.mode_standard(dt=0.005);

In [14]:
all = hoomd.group.all();
hoomd.md.integrate.langevin(group=all, kT=0.2, seed=42);

notice(2): integrate.langevin/bd is using specified gamma values


hoomd.dump.gsd("trajairHCN.gsd", period=2e3, group=all, overwrite=True);

In [15]:
hoomd.run(1e4);

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 19904
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
** starting run **
Time 00:00:27 | Step 2806 / 10000 | TPS 280.547 | ETA 00:00:25
Time 00:00:37 | Step 5503 / 10000 | TPS 269.68 | ETA 00:00:16
Time 00:00:47 | Step 7940 / 10000 | TPS 243.633 | ETA 00:00:08
Time 00:00:57 | Step 10000 / 10000 | TPS 207.426 | ETA 00:00:00
Average TPS: 250.393
---------
-- Neighborlist stats:
152 normal updates / 100 forced updates / 0 dangerous updates
n_neigh_min: 0 / n_neigh_max: 30 / n_neigh_avg: 4.45267
shortest rebuild period: 23
-- Cell list stats:
Dimension: 29, 29, 29
n_min    : 0 / n_max: 17 / n_avg: 0.816106
** run complete **


In [18]:
hoomd.dump.gsd(filename="/home/themotivation/trajairHCN.gsd", period=2e3, group=all, overwrite=True);
box.save("/home/themotivation/Avo_atoms/trajairHCN.gsd")
print("it worked!")

/home/themotivation/anaconda3/lib/python3.5/site-packages/mbuild/compound.py:1768: UserWarning: Guessing that "<AR pos=( 0.5265, 0.2604,-1.3727), 0 bonds, id: 139960532478720>" is element: "Ar"
  warn('Guessing that "{}" is element: "{}"'.format(atom, element))


it worked!


In [25]:
import take
take_snapshot(particles=True, bonds=True, pairs=True, integrators=True, all=True, dtype='float')

ImportError: No module named 'take'